In [ ]:
import matplotlib.pyplot as plt
from mind import mdrun
from numba import njit
import numpy as np
import time

In [ ]:
from mind import correct_distance_pbc
@njit
def calc_rdf(traj, box, grid):
    box_half = box / 2.0
    max_grid = max(grid)
    grid_spacing = grid[1] - grid[0]
    grid_spacing_half = grid_spacing / 2.0
    grid_min = np.min(grid)
    rdf = np.zeros_like(grid)
    N = len(traj[0][0])
    for frame in traj:
        rx, ry, rz = frame
        for i in range(N):
            for j in range(N):
                if i != j:
                    dx = rx[i] - rx[j]
                    dy = ry[i] - ry[j]
                    dz = rz[i] - rz[j]
                    dx, dy, dz = correct_distance_pbc(box, box_half, dx, dy, dz)
                    r = np.sqrt(dx * dx + dy * dy + dz * dz)
                    if r < max_grid + grid_spacing_half:
                        rdf[int((r - grid_min + grid_spacing_half) // grid_spacing)] += 1
    # normalize rdf
    density = N / np.prod(box)
    for i in range(len(rdf)):
        r = grid[i]
        volume_shell = 4 / 3 * np.pi * ((r + grid_spacing_half)**3 -
                                        (r - grid_spacing_half)**3)
        rdf[i] = rdf[i] / volume_shell / density / N / len(traj)
    return rdf

In [ ]:
L = 6.0
box = np.array([L, L, L])
x_ = np.linspace(0., box[0], num=6, endpoint=False)
y_ = np.linspace(0., box[1], num=6, endpoint=False)
z_ = np.linspace(0., box[2], num=6, endpoint=False)
x, y, z = np.meshgrid(x_, y_, z_, indexing='ij')
rx = x.flatten()
ry = y.flatten()
rz = z.flatten()

md_setup = {
    'box': box,
    'start_r': (rx, ry, rz),
    'start_v': None,
    'dt': 0.001,
    'cut_off': 2.5,
    'n_steps': 5000,
    'T': 1.0,
    'tau': 0.1,
    'T_damp': 1,
    'print_every_n_steps': 1000,
    'save_traj_every_n_steps': 1,
    'save_energies_every_n_steps': 1,
}

In [ ]:
r = np.arange(0.5, 2.6, 0.01)
u = 4 * (r**-12 - r**-6)

In [ ]:
trajs, eners, gs = {}, {}, {}
for run in ['LJ', 'tabulated']:
    print(f".. now doing {run} ..")
    if run == 'tabulated':
        md_setup['r_u_table'] = np.array([r, u])

    # MD
    t_start = time.perf_counter()
    trajs[run], eners[run] = mdrun(md_setup)
    t_end = time.perf_counter()
    print(f".. total looping time = {t_end - t_start:.2f} seconds ..")

In [ ]:
# RDF
grid = np.linspace(0.0, np.min(box), 200)
for run in ['LJ', 'tabulated']:
    print(f".. now doing {run} ..")
    t_start = time.perf_counter()
    gs[run] = calc_rdf(trajs[run][1000:], box, grid)  # ignore first x frames
    t_end = time.perf_counter()
    print(f".. total looping time = {t_end - t_start:.2f} seconds ..")

In [ ]:
for run in ['LJ', 'tabulated']:
    plt.plot(grid, gs[run], label=run)
plt.legend()
plt.show()